In [1]:
from lasertram import LaserTRAM, LaserCalc
import lasertram as lt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm


In [2]:
data = pd.read_excel(r"C:\Users\jlubbers\OneDrive - DOI\Research\Coding\laserTRAM-DB\tests\spot_test_timestamp_raw_data.xlsx").set_index('SampleLabel')
data.head()

,timestamp,Time,7Li,24Mg,27Al,29Si,43Ca,48Ti,57Fe,88Sr,138Ba,139La,140Ce,153Eu,208Pb
SampleLabel,,,,,,,,,,,,,,,
BCR-2G_1,2021-03-01 22:08:14,13.24,500.010000,0.0,100.0004,46285.535670,100.0004,0.0000,600.014400,0.0,0.0,0.0,0.0,0.0,100.0004
BCR-2G_1,2021-03-01 22:08:14,163.60,700.019601,0.0,200.0016,43274.778818,0.0000,200.0016,900.032401,0.0,0.0,0.0,0.0,0.0,0.0000
BCR-2G_1,2021-03-01 22:08:14,313.96,600.014400,0.0,400.0064,46586.651171,300.0036,200.0016,1000.040002,0.0,0.0,0.0,0.0,0.0,200.0016
BCR-2G_1,2021-03-01 22:08:14,464.23,800.025601,0.0,100.0004,45683.326387,100.0004,100.0004,1400.078404,0.0,0.0,0.0,0.0,0.0,0.0000
BCR-2G_1,2021-03-01 22:08:14,614.50,600.014400,0.0,300.0036,46185.165445,200.0016,200.0016,700.019601,0.0,0.0,0.0,0.0,0.0,0.0000


In [3]:
samples = data.index.unique().dropna().tolist()

sample = samples[0]

bkgd_interval = (5,10)
keep_interval = (20,50)

spot = LaserTRAM(name = sample)
spot.get_data(data.loc[sample,:])
spot.assign_int_std("29Si")
spot.assign_intervals(bkgd_interval, keep_interval)
spot.get_bkgd_data()
spot.subtract_bkgd()
spot.get_detection_limits()
spot.subtract_bkgd()
spot.normalize_interval()
spot.make_output_report()

In [4]:
spot.output_report

,timestamp,Spot,despiked,bkgd_start,bkgd_stop,int_start,int_stop,norm,norm_cps,7Li,...,29Si_se,43Ca_se,48Ti_se,57Fe_se,88Sr_se,138Ba_se,139La_se,140Ce_se,153Eu_se,208Pb_se
0,2021-03-01 22:08:14,BCR-2G_1,None,5.12408,10.08472,20.00647,50.07226,29Si,2499024.199695,0.001831,...,0.0,6.061414,3.21829,3.000996,1.807116,4.168718,3.689592,3.973801,18.846732,16.604926


In [5]:
spot = LaserTRAM(name = sample)
lt.process_spot(
        spot,
        raw_data = data.loc[sample,:],
        bkgd = (5,10),
        keep = (20,50),
        internal_std = "29Si",
        despike = False,
        output_report = True
        )

In [6]:
bkgd_interval = (5, 8)
keep_interval = (25, 45)

int_std = "29Si"

my_spots = []
print("PROCESSING SPOTS\n")
for sample in tqdm(samples):
    spot = LaserTRAM(name=sample)
    lt.process_spot(
        spot,
        raw_data=data.loc[sample, :],
        bkgd=bkgd_interval,
        keep=keep_interval,
        internal_std=int_std,
        despike=False,
        output_report=True,
    )
    my_spots.append(spot)

print("COMBINING OUTPUT REPORTS\n")
processed_df = pd.DataFrame()
for spot in tqdm(my_spots):
    processed_df = pd.concat([processed_df, spot.output_report])

PROCESSING SPOTS



  0%|          | 0/40 [00:00<?, ?it/s]

COMBINING OUTPUT REPORTS



  0%|          | 0/40 [00:00<?, ?it/s]

In [7]:
processed_df.head()

,timestamp,Spot,despiked,bkgd_start,bkgd_stop,int_start,int_stop,norm,norm_cps,7Li,...,29Si_se,43Ca_se,48Ti_se,57Fe_se,88Sr_se,138Ba_se,139La_se,140Ce_se,153Eu_se,208Pb_se
0,2021-03-01 22:08:14,BCR-2G_1,None,5.12408,8.13056,25.11728,45.11153,29Si,2495372.560199,0.001819,...,0.0,0.850964,0.692265,0.682130,0.707998,0.587471,0.992761,0.845244,1.635309,1.329348
0,2021-03-01 22:09:31,BCR-2G_2,None,5.12417,8.1308,25.11798,45.1115,29Si,2546833.925726,0.001797,...,0.0,0.637530,0.541900,0.497523,0.568544,0.626896,0.598107,0.591970,1.444278,0.865801
0,2021-03-01 22:10:48,ATHO-G_1,None,5.12431,8.13076,25.11669,45.11084,29Si,2530573.062027,0.003787,...,0.0,1.321670,0.467695,0.466671,0.498083,0.508516,0.476264,0.536568,1.509867,6.488146
0,2021-03-01 22:12:05,ATHO-G_2,None,5.12421,8.1309,25.11759,45.1118,29Si,2595616.459733,0.003875,...,0.0,0.829131,0.365662,0.364831,0.423951,0.359817,0.423304,0.393812,1.247436,1.064285
0,2021-03-01 22:13:22,BHVO-2G_1,None,5.12455,8.13149,25.11798,45.11006,29Si,1812510.243358,0.000907,...,0.0,1.093567,1.223828,1.280427,1.207772,1.257159,0.970521,1.854826,1.657233,11.591823


In [8]:
srm_data = pd.read_excel(r"C:\Users\jlubbers\OneDrive - DOI\Research\Coding\laserTRAM-DB\tests\laicpms_stds_tidy.xlsx")


sample
BCR-2G               BCR-2G_1
BCR-2G               BCR-2G_2
ATHO-G               ATHO-G_1
ATHO-G               ATHO-G_2
BHVO-2G             BHVO-2G_1
BHVO-2G             BHVO-2G_2
unknown    unknown_nist-612_1
unknown    unknown_nist-612_2
BCR-2G               BCR-2G_3
ATHO-G               ATHO-G_3
BCR-2G               BCR-2G_4
ATHO-G               ATHO-G_4
BCR-2G               BCR-2G_5
ATHO-G               ATHO-G_5
BCR-2G               BCR-2G_6
ATHO-G               ATHO-G_6
BCR-2G               BCR-2G_7
ATHO-G               ATHO-G_7
BCR-2G               BCR-2G_8
ATHO-G               ATHO-G_8
BHVO-2G             BHVO-2G_3
unknown    unknown_nist-612_3
BCR-2G               BCR-2G_9
ATHO-G               ATHO-G_9
BCR-2G              BCR-2G_10
ATHO-G              ATHO-G_10
BCR-2G              BCR-2G_11
ATHO-G              ATHO-G_11
BCR-2G              BCR-2G_12
ATHO-G              ATHO-G_12
BCR-2G              BCR-2G_13
ATHO-G              ATHO-G_13
BCR-2G              BCR-2G_14
BCR

In [9]:
concentrations = LaserCalc(name="experiment_test")
concentrations.get_SRM_comps(srm_data)
concentrations.get_data(processed_df)
concentrations.set_calibration_standard("BCR-2G")
concentrations.drift_check()
concentrations.get_calibration_std_ratios()
concentrations.set_internal_standard_concentrations(
    concentrations.data["Spot"],
    np.full(concentrations.data["Spot"].shape[0], 71.9),
    np.full(concentrations.data["Spot"].shape[0], 1),
)
concentrations.calculate_concentrations()



In [10]:
concentrations.unknown_concentrations

,timestamp,Spot,7Li,24Mg,27Al,29Si,43Ca,48Ti,57Fe,88Sr,...,29Si_se,43Ca_se,48Ti_se,57Fe_se,88Sr_se,138Ba_se,139La_se,140Ce_se,153Eu_se,208Pb_se
sample,,,,,,,,,,,,,,,,,,,,,
unknown,2021-03-01 22:15:56,unknown_nist-612_1,37.430614,66.744241,11253.642287,336086.996322,92854.309550,462.987407,52.753570,88.168794,...,4848.647411,2038.000577,33.835981,4.166612,1.753629,0.845168,0.957971,0.880149,0.844848,3.635119
unknown,2021-03-01 22:17:13,unknown_nist-612_2,38.075967,64.903258,11200.470510,336086.996322,92529.236470,461.701655,55.730132,87.870494,...,4848.647411,2036.319035,33.748415,2.156204,1.756331,0.836608,0.944231,0.878062,0.850095,3.590856
unknown,2021-03-02 00:32:12,unknown_nist-612_3,38.387653,67.928245,11301.420221,336086.996322,95362.177057,480.710728,53.898815,90.111601,...,4848.647411,2091.016267,35.136249,2.369068,1.804259,0.872097,0.986646,0.903552,0.870817,3.695274
unknown,2021-03-02 02:44:04,unknown_nist-612_4,36.177737,67.208294,11431.675708,336086.996322,93803.808039,475.349004,51.048630,89.440333,...,4848.647411,2073.210173,34.759183,2.633476,1.789987,0.872896,0.967761,0.888008,0.870895,3.687379
unknown,2021-03-02 02:45:22,unknown_nist-612_5,36.275922,68.004839,11303.805177,336086.996322,93664.065054,472.508925,49.301551,88.764919,...,4848.647411,2054.022389,34.539346,2.026981,1.771941,0.850936,0.940056,0.868406,0.857858,3.660260


In [11]:
concentrations.SRM_concentrations

,timestamp,Spot,7Li,24Mg,27Al,29Si,43Ca,48Ti,57Fe,88Sr,...,29Si_se,43Ca_se,48Ti_se,57Fe_se,88Sr_se,138Ba_se,139La_se,140Ce_se,153Eu_se,208Pb_se
sample,,,,,,,,,,,,,,,,,,,,,
ATHO-G,2021-03-01 22:10:48,ATHO-G_1,25.980831,580.427382,65935.976659,353403.141361,12397.257964,1533.414628,23709.618969,96.453399,...,4920.627899,310.403793,112.011356,649.306272,1.909579,10.956875,1.336737,2.583316,0.069747,0.664454
ATHO-G,2021-03-01 22:12:05,ATHO-G_2,26.561203,576.232696,66035.491116,353403.141361,12258.604245,1553.972598,23894.660345,97.302222,...,4920.627899,279.800095,113.422574,650.669017,1.909513,10.743577,1.315682,2.551755,0.066926,0.533147
ATHO-G,2021-03-01 22:39:01,ATHO-G_3,26.130965,593.744416,66729.776409,353403.141361,12713.028897,1587.795506,24336.310933,100.168373,...,4920.627899,285.868415,115.941693,664.467895,1.969560,11.158483,1.375283,2.651594,0.070168,0.556653
ATHO-G,2021-03-01 23:00:51,ATHO-G_4,26.456721,589.486241,66608.572814,353403.141361,12346.345537,1583.271829,24474.287367,99.675454,...,4920.627899,287.007228,115.582582,667.550024,1.955694,11.144340,1.362692,2.615977,0.069373,0.544812
ATHO-G,2021-03-01 23:22:43,ATHO-G_5,26.844400,602.087995,67327.098143,353403.141361,12748.827478,1611.208351,24472.469318,101.061670,...,4920.627899,291.127292,117.676184,667.991085,1.983357,11.343366,1.380904,2.672314,0.071732,0.567860
ATHO-G,2021-03-01 23:44:35,ATHO-G_6,27.287583,598.798143,66844.316139,353403.141361,12560.027700,1610.955995,24817.289420,100.505539,...,4920.627899,291.690155,117.621255,677.307530,1.984063,11.312339,1.376748,2.694900,0.072200,0.566296
ATHO-G,2021-03-02 00:07:45,ATHO-G_7,27.412277,604.449042,67716.076949,353403.141361,12803.576327,1634.125039,24907.019940,102.234365,...,4920.627899,292.143303,119.398437,681.345391,2.029342,11.509738,1.400150,2.728255,0.069808,0.562562
ATHO-G,2021-03-02 00:29:37,ATHO-G_8,26.770341,596.515839,67008.093098,353403.141361,12612.207319,1632.880971,24843.757656,101.078301,...,4920.627899,288.344281,119.286057,678.288856,1.996023,11.343102,1.383144,2.712970,0.070414,0.562239
ATHO-G,2021-03-02 00:54:07,ATHO-G_9,26.978145,600.452897,67009.680987,353403.141361,12610.610941,1625.498113,24724.468250,100.598588,...,4920.627899,288.228527,118.701226,674.803716,1.983432,11.262920,1.379804,2.647950,0.069872,0.549423


In [12]:
concentrations.calibration_standard_stats

,drift_correct,rmse,slope,intercept,mean,std_dev,percent_std_err
7Li,True,5.425889e-05,7.610179e-07,-20.477632,0.001927,0.000092,1.228851
24Mg,True,2.406904e-02,2.667381e-04,-7174.914218,3.207520,0.035592,0.286507
27Al,False,1.228816e-01,7.064479e-04,-18998.422790,12.617315,0.143911,0.294497
29Si,False,1.440625e-12,-2.168404e-19,1.000000,1.000000,0.000000,0.000000
43Ca,True,3.095471e-04,9.168525e-06,-246.694766,0.037086,0.000931,0.647885
48Ti,True,5.018454e-02,7.648076e-04,-20577.365771,4.178212,0.089498,0.553068
57Fe,False,7.246198e-03,4.268943e-05,-1147.548034,1.256366,0.008533,0.175357
88Sr,True,3.377798e-03,1.483062e-04,-3990.676956,0.354957,0.014559,1.059002
138Ba,True,8.855189e-03,4.661271e-04,-12543.030330,0.797164,0.045354,1.469009
139La,True,4.528455e-04,1.787320e-05,-480.950078,0.031140,0.001767,1.464715
